<a href="https://colab.research.google.com/github/Simran-rani78/AI-Pet-Adoption/blob/main/fchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q flask flask-cors pyngrok google-generativeai



In [ ]:
import os
from pyngrok import ngrok
import google.generativeai as genai

# Replace with your real keys below
GOOGLE_API_KEY = "ABC123"
NGROK_AUTH_TOKEN = "DEF456"

# Configure keys
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN
genai.configure(api_key=GOOGLE_API_KEY)

# Set NGROK token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
from flask import Flask, request, render_template_string
from flask_cors import CORS
from pyngrok import ngrok
import google.generativeai as genai

app = Flask(__name__)
CORS(app)

user_identity = "Simran Rani && Abhishek Pandey / 12317008 && 12312235"
model = genai.GenerativeModel("gemini-1.5-flash")

html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>PetBot 🐶</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">

  <style>
    #chat-section.fullscreen {
      position: fixed;
      top: 0;
      left: 0;
      height: 100vh;
      width: 100vw;
      z-index: 50;
      background: white;
      display: flex;
      flex-direction: column;
      padding: 1rem;
    }

    #chat-section.fullscreen #chat {
      flex: 1;
      overflow-y: auto;
      margin-bottom: 1rem;
    }

    #chat-section.fullscreen .input-area {
      display: flex;
      gap: 0.5rem;
      flex-wrap: wrap;
    }

    .hide-main {
      display: none;
    }
  </style>
</head>
<body class="bg-gray-100 flex flex-col h-screen">
  <nav class="bg-pink-600 p-4">
    <div class="container mx-auto flex justify-between">
      <h1 class="text-white text-2xl">PetBot</h1>
      <div>
        <a href="#suggestions" class="text-white hover:text-pink-200 mx-2">Pet Suggestions</a>
        <a href="#chat-section" class="text-white hover:text-pink-200 mx-2">Chat</a>
      </div>
    </div>
  </nav>
  <div class="flex-grow flex items-center justify-center m-8 min-h-[60vh]" id="chat-wrapper">
    <div class="bg-white rounded-lg shadow-lg p-6 w-full max-w-lg h-full" id="chat-section">
      <h2 class="text-center text-3xl text-pink-600 mb-4">🐾 PetBot</h2>
      <div id="chat" class="h-72 overflow-y-auto border border-gray-300 p-2 mb-4 rounded-lg">
        <div class="text-pink-500">🤖 PetBot: Hello! Ask me anything about pets!</div>
      </div>
      <div class="input-area">
        <input type="text" id="userInput" placeholder="Type your question..."
               class="border border-gray-300 rounded-lg p-2 w-full sm:w-4/5"
               onkeydown="if(event.key==='Enter') sendMessage()" />
        <button onclick="sendMessage()"
                class="bg-pink-500 text-white rounded-lg px-4 py-2 hover:bg-pink-600 transition">Send</button>
        <button onclick="endChat()"
                class="bg-red-400 text-white rounded-lg px-4 py-2 hover:bg-red-600 transition">End Chat</button>
      </div>
    </div>
  </div>

  <div id="suggestions" class="bg-white p-6 rounded-lg shadow-lg mx-4 mb-2">
    <h2 class="text-center text-2xl text-pink-600 mb-2">🐾 Pet Suggestions</h2>
    <div class="flex space-x-4 overflow-x-auto snap-x scroll-smooth px-2">
      <div class="border rounded-lg overflow-hidden min-w-[250px] snap-center shrink-0">
        <img src="https://images.pexels.com/photos/416160/pexels-photo-416160.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2" alt="Cat" class="w-full h-48 object-cover">
        <div class="p-4">
          <h3 class="text-lg font-semibold">Cats</h3>
          <p>Perfect for small spaces and independent living.</p>
        </div>
      </div>
      <div class="border rounded-lg overflow-hidden min-w-[250px] snap-center shrink-0">
        <img src="https://images.pexels.com/photos/1108099/pexels-photo-1108099.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2" alt="Dog" class="w-full h-48 object-cover">
        <div class="p-4">
          <h3 class="text-lg font-semibold">Dogs</h3>
          <p>Loyal companions that love to play and exercise.</p>
        </div>
      </div>
      <div class="border rounded-lg overflow-hidden min-w-[250px] snap-center shrink-0">
        <img src="https://images.pexels.com/photos/1228439/pexels-photo-1228439.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2" alt="Rabbit" class="w-full h-48 object-cover">
        <div class="p-4">
          <h3 class="text-lg font-semibold">Rabbits</h3>
          <p>Great for families and can be litter trained.</p>
        </div>
      </div>
      <div class="border rounded-lg overflow-hidden min-w-[250px] snap-center shrink-0">
        <img src="https://images.pexels.com/photos/56733/pexels-photo-56733.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2" alt="Parrot" class="w-full h-48 object-cover">
        <div class="p-4">
          <h3 class="text-lg font-semibold">Parrot</h3>
          <p>Colorful and clever pets that love to mimic sounds and enjoy social interaction.</p>
        </div>
      </div>
      <div class="border rounded-lg overflow-hidden min-w-[250px] snap-center shrink-0">
        <img src="https://images.pexels.com/photos/30371415/pexels-photo-30371415/free-photo-of-clownfish-swimming-in-a-vibrant-aquarium.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2" alt="Rabbit" class="w-full h-48 object-cover">
        <div class="p-4">
          <h3 class="text-lg font-semibold">Fish</h3>
          <p>Peaceful and low-maintenance pets that bring calm and beauty to any space."</p>
        </div>
      </div>
      <div class="border rounded-lg overflow-hidden min-w-[250px] snap-center shrink-0">
        <img src="https://images.pexels.com/photos/793235/pexels-photo-793235.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2" alt="Horse" class="w-full h-48 object-cover">
        <div class="p-4">
          <h3 class="text-lg font-semibold">Horse</h3>
          <p>Strong and intelligent companions, ideal for open spaces and active lifestyles.</p>
        </div>
      </div>
    </div>
  </div>

  <footer class="bg-pink-600 text-white text-center py-4">
    <p>Created by Simran Rani / 12317008 & Abhishek Pandey / 12312235</p>
  </footer>
  <script>
    function sendMessage() {
      const userInput = document.getElementById("userInput");
      const chatBox = document.getElementById("chat");
      const message = userInput.value;
      if (!message.trim()) return;

      chatBox.innerHTML += `<div class="text-gray-800">👤 You : ${message}</div>`;
      userInput.value = "";

      fetch("/chat", {
        method: "POST",
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ question: message })
      })
      .then(res => res.json())
      .then(data => {
        chatBox.innerHTML += `<div class="text-pink-500">🤖 PetBot: ${data.response}</div>`;
        chatBox.scrollTop = chatBox.scrollHeight;
      });
    }

    function endChat() {
      const chatBox = document.getElementById("chat");
      chatBox.innerHTML += `<div class="text-pink-500">🐾 PetBot: Chat ended. Take care of your furry friend! 🐶</div>`;
      document.getElementById("userInput").disabled = true;
    }

    // Enable full-screen chat on "Chat" click
    document.querySelector('a[href="#chat-section"]').addEventListener('click', function (e) {
      e.preventDefault();
      document.getElementById('chat-section').classList.add('fullscreen');
      document.querySelector('nav').classList.add('hide-main');
      document.getElementById('suggestions').classList.add('hide-main');
      document.querySelector('footer').classList.add('hide-main');
    });
  </script>
</body>
</html>

"""
@app.route("/")
def home():
    return render_template_string(html_template)

@app.route("/chat", methods=["POST"])
def chat():
    try:
        user_question = request.json["question"]
        prompt = (
            f"You are a cheerful and helpful pet advisor bot named PetBot, created by {user_identity}. "
            f"You give short, friendly answers with specific recommendations about pet breeds, adoption "
            f"(mentioning actual NGOs or websites when possible), and care tips. Include emojis when it fits.\n\n"
            f":User    {user_question}"
        )
        response = model.generate_content(prompt)
        return {"response": response.text.strip()}
    except Exception as e:
        return {"response": f"Oops! Something went wrong.\nError: {str(e)}"}

# Start server using ngrok (Make sure you have authtoken configured)
public_url = ngrok.connect(5000)
print(f"🌐 PetBot is live at: {public_url}")
app.run(port=5000)

🌐 PetBot is live at: NgrokTunnel: "https://6985-35-231-222-3.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 14:09:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 14:09:30] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 14:09:42] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 15:47:22] "GET / HTTP/1.1" 200 -
